In [1]:
from sklearn.cluster import KMeans #k-means
import numpy as np
import pandas as pd
import cv2
import os
from PIL import Image 
import scipy.misc
import imageio
import glob
import csv

In [3]:
# para crear los directorios nuevos
path="Files/imgs-nuevas/001"
i=0
for folder in os.listdir('Files/png-segments'):
    if(i==4):
        path="Files/imgs_nuevas/"+folder
        try:
            os.makedirs(path)
        except:
            print("Ya existe el directorio"+path)
        path="Files/videos_nuevos/"+folder
        try:
            os.makedirs(path)
        except:
            print("Ya existe el directorio"+path)
    i=i+1

In [4]:
i=0
img_array = []
path="Files/png-segments"
img=cv2.imread("Files/png-segments/001/frame000000_cam0.png")
vec=np.zeros(img.shape[:2])
par=1
path="Files/png-segments"
ñ=1
for folder_name_rep in os.listdir(path): #for para recorrer cada folder
    folder_name="Files/png-segments/"+folder_name_rep
    if(i==4):
        for frame in os.listdir(folder_name):#for para recorrer los frames dentro de cada folder
            if(par==1):
                frame_uno=folder_name+"/"+frame
                frame_uno = np.array(cv2.imread(frame_uno),np.float32) #aquí está el frame
                par=2
            else:
                frame_dos=folder_name+"/"+frame                
                frame_dos = np.array(cv2.imread(frame_dos),np.float32) #aquí está el frame
                flow=np.array(abs(frame_dos[:240,:]-frame_uno[:240,:]),np.uint8)#resultado del movimiento, diferencia de frames

    #            max_value=np.max(flow)
    #             for i in range(flow.shape[0]):
    #                 for j in range(flow.shape[1]):
    #                     if(flow[i,j,0]<=max_value-100):
    #                         flow[i,j]=0
                frame_uno=frame_dos

                img_array.append(flow) #array con las imagenes resultantes
                path='Files/imgs_nuevas/'+folder_name_rep+'/'
        for i in range(len(img_array)):        #apartir de aquí se producen y guardan las imgs denetro de cada uno de sus 
    #folders respectivos
            try:
                imageio.imwrite((path+str(i)+'img.jpg'), img_array[i])
            except:
                print("No se puede crear la img: "+path+img_array[i])
        print('Lista la carpeta: '+str(ñ))
        ñ=ñ+1
    #     break
    i=i+1
    
print('salió del for')

Lista la carpeta: 1
salió del for


In [ ]:
# crear videos a partir de los frames optimizados
path="Files/videos_nuevos/"
for folder in os.listdir(path):
    print(folder)
    img_array = []
    try:
        for filename in glob.glob('Files/imgs_nuevas/'+folder+'/*.jpg'):
            img = cv2.imread(filename)
            height, width, layers = img.shape
            size = (width,height)
            img_array.append(img)


        out = cv2.VideoWriter(path+folder+'/'+folder+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)

        for i in range(len(img_array)):
            out.write(img_array[i])
    except:
        print('No se puede crear el video')
#     break
out.release()

# Dense Optical Flow y k-means

In [5]:
def create_words_counter_per_sentence(trainFileName):
    train_data = pd.read_csv(trainFileName,delimiter=";")
    sentences_train = [sentence[:len(sentence) -1 ] for sentence in train_data.orth]
    sentences = sentences_train
    numberofWords = []
    for sentence in sentences:
        sentence = sentence.split()
        counter = 0
        for  word in sentence:   
            word = word.lower()
            if((word != "[silence]" and word != "[silence")):
#                 print (word,counter)
                counter+=1
        numberofWords.append(counter)
    del train_data
    del sentences
    del sentences_train
    return numberofWords

In [6]:
# para determinar la duración de los videos
def getLength(filename):
    v=cv2.VideoCapture(filename)
    v.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
    segs = v.get(cv2.CAP_PROP_POS_MSEC)
    segs=segs/1000
    return segs
def duraciones_de_videos(path):
    duraciones=[]    
    for folder in os.listdir(str(path)):
        for file in os.listdir(path+folder):
            video=path+folder+"/"+file
            lg=getLength(video)
            duraciones.append(lg)
    return duraciones

In [12]:
# Dense Optical Flow in OpenCV y creo el vector de vectores
filename = "../../corpus/train.sentences.pronunciations.multi.translations.csv"
totalWords = create_words_counter_per_sentence(filename) 
videos="Files/videos_nuevos/"
duraciones = duraciones_de_videos(videos)

path='Files/videos_nuevos/'
array_vecs=[]
array_imgs=[]
array_movs=[]
vector_de_frames=[]
posicion=0
contador=0
for folder in os.listdir(path):    
    for file in os.listdir(str(path+folder)):
        print(file)
        cap = cv2.VideoCapture(str(path+folder+'/'+file))
        ret, frame1 = cap.read()
        prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
        hsv = np.zeros_like(frame1)
        hsv[...,1] = 255

        while(1):
            ret, frame2 = cap.read()
            try:
                next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
                flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                
                mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])

                array_vecs.append(([ang]))

                hsv[...,0] = ang*180/np.pi/2
#                 hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
#                 print(np.array(hsv[...,0]).shape) #hsv[...,0] es la que posee la información en ángulos del mov
#                 bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
#                 bgr2=np.array(cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR),np.uint32)
#                 cv2.imshow('frame2',bgr)
                vect_temp=np.array(hsv[...,0]).flatten()
                vector_de_frames=np.concatenate((vector_de_frames,vect_temp),axis=0)
    
                contador=contador+1
    
#                 k = cv2.waitKey(30) & 0xff
#                 if k == 27:
#                     break
#                 elif k == ord('s'):
#                     cv2.imwrite('opticalfb.png',frame2)
#                     cv2.imwrite('opticalhsv.png',bgr)
                prvs = next
            except:
                break    
#         print(len(array_vecs))
#             se dividirá el vector de frames en cuantas palabras tenga el video
        tamaño_vec=len(vector_de_frames)
        with open('putos_todos.csv', 'w', newline='') as csvfile:
            filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            for i in range(totalWords[posicion]):
                comienzo=int((tamaño_vec/totalWords[posicion])*(i))
                fin=int((tamaño_vec/totalWords[posicion])*(i+1))
#                 array_movs.append(vector_de_frames[comienzo:fin])
                print(vector_de_frames[comienzo:fin])
                print (vector_de_frames.shape)
                print(type(vector_de_frames))
                filewriter.writerow(vector_de_frames[comienzo:fin])
    vector_de_frames=[]
    posicion=posicion+1    
    if(posicion==1):
        break
cap.release()
cv2.destroyAllWindows()

001.avi
[0. 0. 0. ... 0. 0. 0.]
(7096320,)
<class 'numpy.ndarray'>
[ 0.  0.  0. ... 90. 90. 90.]
(7096320,)
<class 'numpy.ndarray'>
[ 0.  0.  0. ... 90. 90. 90.]
(7096320,)
<class 'numpy.ndarray'>


In [13]:
with open('putos_todos.csv', 'r') as f:
    reader = csv.reader(f)
 
    # read file row by row
    for row in reader:
        print (row[:10])

['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0']
['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0']
['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0']


In [14]:
np.array(array_movs).shape


(3, 2365440)

In [ ]:
# para converitr a vector las matrices y luego concadenarlas en un solo vector

a=np.zeros((10,10))
a=a.flatten()
b=np.ones((3,5))
b=b.flatten()
c=np.concatenate((a,b), axis=0)
c.shape

In [ ]:
# para crear los directorios para los videos cortados
try:
    path="Files/png-segments-cutted"
    os.makedirs(path)
except:
    print("Ya existe el directorio"+path)
counter=0
# for folder in os.listdir('Files/png-segments'):
#     path="Files/png-segments-cutted/"+folder
    
#     for i in range(totalWords[counter]):
# #         print(i)
#         try:
#             os.makedirs(path+"part_"+str(i))
#         except:
#             print("Ya existe el directorio "+path+"part_"+str(i))
#     counter=counter+1
print(len(totalWords))
print(len(os.listdir('Files/png-segments')))

In [ ]:
print(len(array_movs))
ant=0
for i in array_movs:
    print(len(i))
minin=100000
for i in array_movs:
    if(minin<len(i)):
        minin=len(i)
        
print("Minimo  -----  "+str(minin))

In [ ]:

tamaño_vec=len(vector_de_frames)
array_movs=[]
for i in range(3):
    comienzo=int((tamaño_vec/3)*(i))
    fin=int((tamaño_vec/3)*(i+1))
    array_movs.append(vector_de_frames[comienzo:fin])
len(array_movs)

In [ ]:
n=np.zeros(1000)
temP=[]
# temP=np.array(temP)
for i in range(3):
    comienzo=int((len(n)/3)*(i))
    fin=int((len(n)/3)*(i+1))
    temP.append(n[comienzo:fin])
int(np.array(temP[2]).shape[0])+int(np.array(temP[1]).shape[0])+int(np.array(temP[0]).shape[0])

In [ ]:
totalWords[0]

In [ ]:
array_angs=[]
# print(hsv[:,:,0].shape)
i=0
for fila in hsv[:,:,0]:
#     print("Fila=" + str(i))
    if (np.max(fila)!=0):
        array_angs.append(fila)
#         print(fila)
    
    i=i+1
np.array(array_angs).shape

In [ ]:
n_angles = len(array_vecs)
angle = array_vecs[8][0]
print(angle.shape)

for ang in angle:
#     if((ang!=0) and (ang != 0.0)):
#         print(ang)
    print(ang)

In [ ]:
n=np.array(hsv)
print(type(n))
print(type(n[0]))
n.shape

In [ ]:
len(array_vecs)

In [ ]:
k=10
kmeans = KMeans(n_clusters=k, random_state=0).fit(n[:,:,0])
print(kmeans.labels_)
print(kmeans.labels_.shape)

In [ ]:
# img =n[:,:,2]
img = hsv[...,2]
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(img)
plt.colorbar()
data_i= np.reshape(())

In [ ]:
i = 10
elemento = array_vecs[i]
dire = elemento[0]
mag.shape


In [ ]:
dire.shape

In [ ]:
plt.imshow(mag)

In [ ]:
plt.imshow(dire)

In [ ]:
import numpy as np
x = np.cos(dire)
y = np.sin(dire)
x

In [ ]:
plt.imshow(x + y)